# API de OpenWeatherMap para data histórica de contaminación del aire

## Parámetros de la API:

* `lat`: Latitud de la ciudad.

* `lon`: Longitud de la ciudad.

* `start`: Fecha de inicio en tiempo Unix (zona horaria UTC).

* `end`: Fecha de fin en tiempo Unix (zona horaria UTC).

* `appid`: Clave única, que puedes encontrar en la página de tu cuenta denominada `API key`.

In [1]:
# Importar librerias
import pandas as pd
import requests
import time
from datetime import datetime, timedelta
from sqlalchemy import create_engine

In [2]:
# Importar las funciones desde functions.py
from functions import api_key_from_file, extract_data_from_api, transform_data, pwd_from_file, load_data_to_redshift

In [3]:
# Leer la clave de API
api_key = api_key_from_file()

In [4]:
# Definir los parámetros necesarios
api_key = api_key
lat = "-37.2463"
lon = "-73.3175"
end_date = datetime.now()
start_date = end_date - timedelta(days=18*30)

## Campos en la respuesta de la API:

* `coord`: Coordenadas de la ubicación especificada (latitud, longitud).

* `dt`: Fecha y hora en formato Unix (UTC).

* `main.aqi`: Índice de Calidad del Aire (Air Quality Index). Valores posibles:
  - 1 = Bueno
  - 2 = Aceptable
  - 3 = Moderado
  - 4 = Malo
  - 5 = Muy malo

* `components.co`: Concentración de CO (Monóxido de Carbono), μg/m³.
* `components.no`: Concentración de NO (Monóxido de Nitrógeno), μg/m³.
* `components.no2`: Concentración de NO2 (Dióxido de Nitrógeno), μg/m³.
* `components.o3`: Concentración de O3 (Ozono), μg/m³.
* `components.so2`: Concentración de SO2 (Dióxido de Azufre), μg/m³.
* `components.pm2_5`: Concentración de PM2.5 (Partículas Finas), μg/m³.
* `components.pm10`: Concentración de PM10 (Partículas Gruesas), μg/m³.
* `components.nh3`: Concentración de NH3 (Amoníaco), μg/m³.

In [5]:
# Extraer datos de la API
raw_data = extract_data_from_api(api_key, lat, lon, start_date, end_date)
raw_data

{'coord': {'lon': -73.3175, 'lat': -37.2463},
 'list': [{'main': {'aqi': 1},
   'components': {'co': 186.92,
    'no': 0,
    'no2': 0.79,
    'o3': 14.48,
    'so2': 0.05,
    'pm2_5': 1.32,
    'pm10': 1.57,
    'nh3': 0.17},
   'dt': 1675990800},
  {'main': {'aqi': 1},
   'components': {'co': 183.58,
    'no': 0,
    'no2': 0.7,
    'o3': 14.84,
    'so2': 0.04,
    'pm2_5': 0.79,
    'pm10': 1.01,
    'nh3': 0.15},
   'dt': 1675994400},
  {'main': {'aqi': 1},
   'components': {'co': 183.58,
    'no': 0,
    'no2': 0.61,
    'o3': 16.63,
    'so2': 0.03,
    'pm2_5': 0.53,
    'pm10': 0.7,
    'nh3': 0.13},
   'dt': 1675998000},
  {'main': {'aqi': 1},
   'components': {'co': 183.58,
    'no': 0,
    'no2': 0.5,
    'o3': 18.6,
    'so2': 0.03,
    'pm2_5': 0.5,
    'pm10': 0.64,
    'nh3': 0.13},
   'dt': 1676001600},
  {'main': {'aqi': 1},
   'components': {'co': 181.91,
    'no': 0,
    'no2': 0.4,
    'o3': 20.21,
    'so2': 0.03,
    'pm2_5': 0.5,
    'pm10': 0.63,
    'nh3': 0.

In [6]:
# Transformar los datos
df_agrupado = transform_data(raw_data)
df_agrupado

,date,aqi,co,no,no2,o3,so2,pm_2_5,pm_10,nh3
0,2023-02-09,1.000000,185.250000,0.000000,0.745000,14.660000,0.045000,1.055000,1.290000,0.160000
1,2023-02-10,1.000000,181.495000,0.064167,0.399167,18.835000,0.054583,1.197917,1.825000,0.097083
2,2023-02-11,1.000000,170.510833,0.053750,0.368750,26.203750,0.092500,2.274167,4.873333,0.096667
3,2023-02-12,1.000000,181.842917,0.050417,0.480833,21.025417,0.079583,3.014583,7.667917,0.162500
4,2023-02-13,1.000000,183.651250,0.104167,0.713333,17.174167,0.066250,2.967917,4.705833,0.193750
...,...,...,...,...,...,...,...,...,...,...
535,2024-07-29,1.041667,310.698750,0.346250,2.553333,41.648750,0.057917,3.558333,6.138333,0.727917
536,2024-07-30,1.041667,310.004167,0.071667,2.886667,35.047083,0.318333,5.675000,10.116667,0.444167
537,2024-07-31,2.000000,254.930000,0.002083,0.652083,64.269167,0.694167,4.266667,6.524583,0.061667
538,2024-08-01,2.000000,222.939583,0.000417,0.207083,66.459167,0.337083,1.860000,2.674167,0.004583


In [7]:
# Leer la contraseña
password = pwd_from_file()

In [8]:
#  Definir las credenciales de Redshift
redshift_credentials = {
    'host': 'data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com',
    'port': '5439',
    'user': 'rincybarra_coderhouse',
    'password': password,
    'dbname': 'data-engineer-database'
}

In [9]:
# Definir el nombre de la tabla en Redshift
table_name = 'air_pollution'

# Cargar los datos en Redshift
load_data_to_redshift(df_agrupado, table_name, redshift_credentials)

La inserción fue exitosa
('2023-02-09', 1.0, 185.25, 0.0, 0.745, 14.66, 0.045, 1.055, 1.29, 0.16, datetime.datetime(2024, 8, 3, 1, 59, 13))
('2023-02-10', 1.0, 181.495, 0.0641666666666667, 0.399166666666667, 18.835, 0.0545833333333333, 1.19791666666667, 1.825, 0.0970833333333333, datetime.datetime(2024, 8, 3, 1, 59, 13))
('2023-02-11', 1.0, 170.510833333333, 0.05375, 0.36875, 26.20375, 0.0925, 2.27416666666667, 4.87333333333333, 0.0966666666666667, datetime.datetime(2024, 8, 3, 1, 59, 13))
('2023-02-12', 1.0, 181.842916666667, 0.0504166666666667, 0.480833333333333, 21.0254166666667, 0.0795833333333333, 3.01458333333333, 7.66791666666667, 0.1625, datetime.datetime(2024, 8, 3, 1, 59, 13))
('2023-02-13', 1.0, 183.65125, 0.104166666666667, 0.713333333333333, 17.1741666666667, 0.06625, 2.96791666666667, 4.70583333333333, 0.19375, datetime.datetime(2024, 8, 3, 1, 59, 13))
('2023-02-14', 1.0, 184.415416666667, 0.0595833333333333, 0.447083333333333, 29.0570833333333, 0.140833333333333, 2.26416

In [10]:
# Mostrar el DataFrame resultante
df_agrupado

,date,aqi,co,no,no2,o3,so2,pm_2_5,pm_10,nh3
0,2023-02-09,1.000000,185.250000,0.000000,0.745000,14.660000,0.045000,1.055000,1.290000,0.160000
1,2023-02-10,1.000000,181.495000,0.064167,0.399167,18.835000,0.054583,1.197917,1.825000,0.097083
2,2023-02-11,1.000000,170.510833,0.053750,0.368750,26.203750,0.092500,2.274167,4.873333,0.096667
3,2023-02-12,1.000000,181.842917,0.050417,0.480833,21.025417,0.079583,3.014583,7.667917,0.162500
4,2023-02-13,1.000000,183.651250,0.104167,0.713333,17.174167,0.066250,2.967917,4.705833,0.193750
...,...,...,...,...,...,...,...,...,...,...
535,2024-07-29,1.041667,310.698750,0.346250,2.553333,41.648750,0.057917,3.558333,6.138333,0.727917
536,2024-07-30,1.041667,310.004167,0.071667,2.886667,35.047083,0.318333,5.675000,10.116667,0.444167
537,2024-07-31,2.000000,254.930000,0.002083,0.652083,64.269167,0.694167,4.266667,6.524583,0.061667
538,2024-08-01,2.000000,222.939583,0.000417,0.207083,66.459167,0.337083,1.860000,2.674167,0.004583
